<a href="https://colab.research.google.com/github/longjae/raft_rag/blob/master/no_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai datasets

In [ ]:
!pip install -U datasets

In [ ]:
import datasets
from huggingface_hub import login
from datasets import load_dataset

In [ ]:
# 1. Hugging Face에서 데이터셋 불러오기
dataset = load_dataset("longjae/klue-mrc-bge-m3")

# 2. train split을 데이터프레임으로 변환
df = dataset['train'].to_pandas()

# 3. 데이터프레임 확인
print(df.head())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/640 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/77.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10434 [00:00<?, ? examples/s]

                                         title news_category     source  \
0                       제주도 장마 시작 … 중부는 이달 말부터            종합   hankyung   
1      부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정            경제    acrofan   
2                                     나루세 요시히사          None  wikipedia   
3                               편의점 휩쓴 ‘맛집 라면’          생활경제   hankyung   
4  사모펀드 ACP 아난드 프라카시 전무 ...“아시아 친환경 투자 황금기 온다”            기획   hankyung   

                                             context  \
0  올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...   
1  부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...   
2  시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...   
3  유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...   
4  “앞으로 5년 안에 아시아 친환경·신재생에너지 투자의 황금기가 도래할 것입니다.”기...   

                              question  question_type  is_impossible  \
0  북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?              1          False   
1          지능형 생산자동화 기반기술을 개발중인 스타트업은?              2          False

In [ ]:
# 임의 샘플링
df = df[1000:1500].reset_index(drop=True)

In [ ]:
samples = []
for context, negative_samples in zip(df['context'].to_list(), df['negative_samples'].to_list()):
  samples.append([context] + list(negative_samples))

In [ ]:
# 프롬프트에 검색 문서 넣는 연습
for idx, s in enumerate(samples[5]):
  print('문서' + str(idx + 1) + ':' + s)
  print('---')

문서1:정부의 8·28 부동산 활성화 대책에 대한 기대로 은행 가계대출이 사상 최고치를 경신했다.13일 한국은행에 따르면 10월 말 예금은행 가계대출 잔액은 474조4000억원으로 전달보다 2조5000억원 증가했다. 지난 6월 4조8000억원 늘어난 뒤 증가세가 잠시 주춤했지만 지난달 다시 큰 폭으로 늘어났다. 은행이 주택금융공사로 넘긴 모기지론 양도분까지 포함하면 10월 한 달간 은행 가계대출은 2조8000억원 급증했다.김정현 한은 금융시장팀 차장은 “취득세 영구 인하 소급 적용에 대한 기대로 주택 거래량이 크게 늘어나면서 주택담보대출(모기지론 양도 포함)이 2조1000억원 늘었다”고 설명했다. 서울시 아파트 거래량은 8월 2800가구에 그쳤으나 9월 4200가구에 이어 10월 7600가구로 증가했다.마이너스통장대출 등 기타 대출도 7000억원 늘었다. 지난 9월 추석 상여금이 통장에 들어오면서 기타 대출은 1000억원 줄었으나 지난달 다시 증가세로 돌아섰다. 한편 10월 은행의 기업대출 증가세는 둔화됐다. 9월 증가액은 5조8000억원이었으나 10월에는 4조1000억원에 그쳤다. 대기업이 1조3000억원, 중소기업이 2조8000억원 각각 늘었다.
---
문서2:박근혜 대통령이 최근 담보 위주의 대출만 취급하는 ‘금융권 보신주의’를 지적하면서 은행들의 중소기업 대출 실적에 관심이 모아지고 있다. 원금과 이자 연체 등 부실 대출이 많았던 국민은행과 우리은행이 건전성 강화 등을 이유로 몸을 사리는 동안, 농협은행과 신한은행이 공격적인 영업으로 치고 나간 모양새다. KT ENS 협력사에 대한 부실대출 등으로 올초 몸살을 앓았던 하나은행도 뒤늦게 중소기업 대출 확대에 나서고 있다.○국민·우리 대출 증가폭 최저국민은행의 중소기업 대출 잔액은 지난달 말 기준으로 68조4155억원이었다. 지난해 말보다 0.9% 늘어나는 데 그쳤다. 우리은행은 59조1265억원으로 같은 기간 증가율이 1.5%에 불과했다. 통상 대출자산 증가율 목표치를 국내총생산(GDP) 증가율보다 조금 

In [ ]:
user_prompts = []
for sample in samples:
  user_prompt = ''
  for idx, s in enumerate(sample):
    user_prompt = user_prompt + '문서' + str(idx + 1) + ':' + s + '\n'
    user_prompt = user_prompt + '---\n'
  user_prompts.append(user_prompt)

In [ ]:
system_prompt = """당신은 주어진 5개의 검색 문서로부터 답변이 불가능한 질문 5개를 생성해야 합니다.

### 구체적인 지시사항
1. 문서를 5개 드리겠습니다. 각 문서들과 연관된 질문을 하나씩 생성하십시오. 문서1, 문서2, 문서3, 문서4, 문서5 순서대로 합니다.
2. 질문은 각 문서와 연관된 질문이어야 합니다.
3. 답변의 순서는 연관 문서:, 질문:, 판단: 이 세 가지이며 5개의 문서가 주어지므로 총 5번 써야합니다.
4. 중요한 점은 생성된 각 질문은 5개의 그 어떠한 문서로도 답변이 불가능해야 한다는 것입니다.
5. 질문 작성 후에는 추가적인 코멘트는 작성하지 마십시오.
6. 답변의 형식은 다음과 같습니다. 연관 문서에서 질문을 생성하는데 참고한 문서를 적고 그 다음에 질문: 다음에 실제 질문을 생성하세요.
7. 판단: 과정에서 각 질문이 정말로 5개의 문서로 답변이 불가능한지 다시 한 번 판단하고 특정 문서로 답변이 가능한 것으로 판단되면 => False를 적고 답변이 불가능한게 맞다면 => True를 적으세요.
8. => False와 => True 다음에 추가적인 코멘트는 금지합니다.
9. 질문은 짧게 작성하십시오. 예를 들어보겠습니다.
###
ex) 질문: 원종이 설립된 이후 해인사에서 어떤 특별한 행사나 의식이 처음으로 개최되었는지 알려주시겠습니까?
위와 같이 길게 작성하지 마십시오.
ex) 질문: 원종이 설립되고 해인사에서 처음 개최된 행사를 알려줘
위와 같이 짧게 작성하십시오.
###
10. 각 질문은 존댓말과 반말을 교차하면서 사용하십시오. 반말을 했다면 다음 질문은 가급적 존댓말을 하는 식입니다. 최대한 지켜주십시오.
11. 답변 형식은 다음과 같스빈다.
ex)
답변:
연관 문서: 문서1
질문: (문서1과 연관된 질문이지만 5개의 문서 그 어떤 것을 사용하더라도 답변이 불가능한 질문을 실제로 작성하세요.)
판단: 해당 질문은 문서 3에 연관된 내용 ~~가 있어서 답변이 가능합니다. => False

연관 문서: 문서2
질문: (문서2와 연관된 질문이지만 5개의 문서 그 어떤 것을 사용하더라도 답변이 불가능한 질문을 실제로 작성하세요.)
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

...중략...

연관 문서: 문서5
질문: (문서5와 연관된 질문이지만 5개의 문서 그 어떤 것을 사용하더라도 답변이 불가능한 질문을 실제로 작성하세요.)
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True
"""

In [ ]:
from tqdm import tqdm
import openai

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
client = openai.OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
# 5개로 테스트
result_lst = []
for user_prompt in tqdm(user_prompts[:5]):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '답변:'}
      ]
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 5/5 [00:31<00:00,  6.23s/it]


In [ ]:
print(result_lst[0])

답변:
연관 문서: 문서1  
질문: 금강경의 공(空) 개념이 어떻게 사람들에게 적용될 수 있는지 설명해줄 수 있어?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서2  
질문: 원종이 설립되기 전에 어떤 이벤트가 있었는지 알려줘.  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서3  
질문: 인문여행 중에 설총이 남긴 특정한 교훈이 뭐였는지 말해줘.  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서4  
질문: 부처님오신날에 어떤 음식이 주로 준비되는지 알아?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서5  
질문: 간화선과 위파사나의 차이점을 구체적으로 설명해줄 수 있어?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True


In [ ]:
print(result_lst[1])

연관 문서: 문서1  
질문: 디턴 교수의 첫번째 책 제목은 무엇인가요?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  

연관 문서: 문서2  
질문: 디턴 교수가 수상 발표 전에 어떤 음식점을 방문했는지 알아?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  

연관 문서: 문서3  
질문: 디턴 교수의 친구가 몇 년 전에 어떤 저서를 썼는지 말해줄 수 있어?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  

연관 문서: 문서4  
질문: 올해 한국에서 가장 인기 있는 부동산 투자 지역은 어디야?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  

연관 문서: 문서5  
질문: 크루그먼 교수가 대학원 시절에 어떤 강의를 들었는지 알아?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  


In [ ]:
print(result_lst[2])

답변:
연관 문서: 문서1  
질문: 황소자리 T가 발견된 후 어떤 변화를 겪었는지 알려줘?  
판단: 해당 질문은 문서 1에서만 설명되고 있으며, 다른 문서로는 답변이 불가능하다. => False  

연관 문서: 문서2  
질문: 하버드의 계산수들이 발견한 다른 천문학적 사실들을 알고 싶어.  
판단: 해당 질문은 문서 2에만 관련이 있고, 다른 문서는 그 내용을 다루지 않아 답변이 불가능하다. => False  

연관 문서: 문서3  
질문: 유로파 탐사에서 제일 먼저 사용할 과학장비는 뭐야?  
판단: 해당 질문은 문서 3에 국한되어 있으며, 다른 문서로는 답변할 수 없다. => False  

연관 문서: 문서4  
질문: 닐 슈빈이 발견한 화석의 구체적인 위치를 알려줄 수 있어?  
판단: 해당 질문은 문서 4에만 관련이 있으며, 다른 문서에서는 답변이 불가능하다. => False  

연관 문서: 문서5  
질문: 아편굴에서 범죄로 드러난 다른 사건이 뭐가 있는지 알고 싶어.  
판단: 해당 질문은 문서 5와 관련이 있지만, 다른 문서에서의 정보로는 답변이 불가능하다. => False


In [ ]:
print(result_lst[3])

답변:
연관 문서: 문서1  
질문: 온세미컨덕터의 1200V 애플리케이션용 모듈의 가격은 얼마인가요?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  

연관 문서: 문서2  
질문: EVL400W-EUPL7 보드의 출시일은 언제인가요?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  

연관 문서: 문서3  
질문: LT8491의 사용자는 몇 개인가요?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  

연관 문서: 문서4  
질문: XDPL8219의 특정 애플리케이션에 대한 선정 이유는 무엇인가요?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  

연관 문서: 문서5  
질문: TLE9012AQU의 부품원가는 얼마인가요?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  


In [ ]:
print(result_lst[4])

답변:
연관 문서: 문서1
질문: 삼성전자의 주가가 언제 150만원에 도달했는지 알 수 있을까?
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서2
질문: CJ E&M이 2019년에 배급한 영화 목록을 알려줘
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서3
질문: KB금융지주가 발행했던 최초의 회사채 규모는 얼마였나요?
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서4
질문: 넥슨의 첫번째 게임 출시일은 언제인가요?
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True

연관 문서: 문서5
질문: 삼성증권의 IPO사업부가 언제 처음 설립됐는지 말해줄 수 있어?
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True


In [ ]:
result_lst = []
for user_prompt in tqdm(user_prompts):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '답변:'}
      ]
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 500/500 [51:53<00:00,  6.23s/it]


In [ ]:
df['no_answer_question'] = result_lst

In [ ]:
df

,title,news_category,source,context,question,question_type,is_impossible,answer_text,answer_start,negative_samples,no_answer_question
0,"“금강경, 중학생도 읽을 수 있게 풀어썼죠”",문화/TV,hankyung,“‘금강경’은 저에게도 늘 어려웠어요. 우리 불교가 관념적 해석에만 치우쳐 대중에게...,대중이 불교에 친숙해질 수 있도록 일감 스님이 주최한 행사의 이름은?,1,False,‘내비둬 콘서트’,147,[이러한 배경 속에 1908년 3월에 불교계 대표 52인이 한성부 원흥사에 모여서 ...,답변:\n연관 문서: 문서1 \n질문: 금강경을 쓴 후 일감 스님이 어떤 복잡한 ...
1,“디턴 불평등論 왜곡 없었다”...‘위대한 탈출’ 번역 출판한 한경BP 밝혀,종합,hankyung,올해 노벨 경제학상 수상자로 선정된 앵거스 디턴 미국 프린스턴대 교수의 저서 ‘위대...,올해 노벨 경제학상 수상자의 저서와 공통된 소재를 다룬 책은?,2,False,피케티의 ‘21세기 자본’,718,[“가난한 이들이 어떻게 행동하는지 연구하고 어떻게 하면 그들이 좋은 삶을 누릴 수...,답변:\n연관 문서: 문서1 \n질문: 디턴 교수의 개인적 의견이 어떤가요? \...
2,허빅-아로천체,None,wikipedia,"허빅-아로천체는 19세기 말에 버넘이 처음 발견했는데, 그가 릭 관측소에서 91 c...","버넘이 허빅-아로천체를 찾아냈을 때, 사용한 물건은?",1,False,91 cm 구경의 굴절망원경,42,[하버드의 계산수들(Harvard Computers)은 미국 천문학자 에드워드 찰스...,답변:\n연관 문서: 문서1 \n질문: 허빅-아로 천체가 발견된 이후 어떤 다른 ...
3,"온세미컨덕터, 산업용 모터 드라이브 애플리케이션 포트폴리오 확대",테크,acrofan,온세미컨덕터(Nasdaq: ON)가 기업들이 특정 설계 문제를 해결하도록 돕기 위해...,NCP730이 동작하는 전압은 최대 몇 볼트인가?,1,False,38V,1503,"[ST마이크로일렉트로닉스(STMicroelectronics, 이하 ST)가 EVL4...",답변: \n연관 문서: 문서1 \n질문: 이 제품의 사용 사례로 어떤 산업 분야...
4,원高·삼성 ‘쇼크’… 증시 불안한 출발,종합,hankyung,코스피지수가 44.15포인트 떨어지는 등 새해 첫 거래일 한국 증시가 호된 신고식을...,13%가량 자사 예상 주가를 줄인 기업은?,1,False,BNP파리바,394,[상장사 10곳 중 6곳의 올 1분기 영업이익 전망치가 지난달 말보다 하향 조정됐다...,답변:\n연관 문서: 문서1 \n질문: 코스피 지수가 최근 몇 년간 가장 많이 하...
...,...,...,...,...,...,...,...,...,...,...,...
495,볼펜,None,wikipedia,볼펜은 일회용 모델과 리필 모델로 생산된다. 리필을 사용하면 볼펜 및 소켓을 포함한...,일부 볼펜에서 잉크가 비교적 빨리 건조되기 위해 사용되는 것은?,2,False,하이브리드 잉크 배합물,530,[볼펜 잉크는 일반적으로 염료가 25-40 % 정도 포함 된 페이스트이다. 염료는 ...,답변:\n연관 문서: 문서1\n질문: 볼펜이 어떤 색상의 잉크를 사용할 때 가장 잘...
496,뇌전증,None,wikipedia,"뇌전증 발작은 뇌피질세포의 기능 이상에 의해 발생하는 증상이므로, 신경세포의 기능 ...",뇌전증 발작은 어떤 세포의 기능적 문제로 발생하는 증상인가요?,1,False,뇌피질세포,8,"[간질(癎疾) 또는 전간(癲癎)이라고도 하며, 과거에는 간질이 정식 명칭으로 사용되...",답변:\n연관 문서: 문서1 \n질문: 뇌전증 환자의 높은 발작 빈도를 유발하는 ...
497,뇌전증,None,wikipedia,"간질(癎疾) 또는 전간(癲癎)이라고도 하며, 과거에는 간질이 정식 명칭으로 사용되었...",뇌전증이 가장 많이 발병하는 때는 언제까지인가?,1,False,생후 1년,688,"[뇌전증 발작은 뇌피질세포의 기능 이상에 의해 발생하는 증상이므로, 신경세포의 기능...",답변:\n연관 문서: 문서1\n질문: 뇌전증 환자의 통증 완화 방법이 뭐야?\n판단...
498,힐튼과 맞먹는 에어비앤비 기업가치,국제,hankyung,세계 최대 숙박 공유 서비스기업 에어비앤비가 최근 15억달러(약 1조6800억원) ...,기업 가치가 가장 큰 비상장 기업의 이름은?,1,False,샤오미,292,[공정거래위원회가 자산 5조원 이상 대기업집단의 계열사 중 자산 100억원 미만 비...,답변:\n연관 문서: 문서1\n질문: 에어비앤비의 설립자인 브라이언 체스키가 어떤 ...


## 데이터 정제

In [ ]:
import re
import random

In [ ]:
false_samples = []
for sample in df['no_answer_question'].to_list():
  if '=> False' in sample:
    false_samples.append(sample)

print('False가 발생한 샘플의 수:', len(false_samples))

False가 발생한 샘플의 수: 59


In [ ]:
print(df['no_answer_question'].to_list()[15])

답변:
연관 문서: 문서1  
질문: 서울 강남에서 가장 비싼 건물의 명칭이 뭐야?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  

연관 문서: 문서2  
질문: 이번 년도에 개통된 철도 노선의 종류는 뭐냐?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  

연관 문서: 문서3  
질문: 마드리드 지하철 9호선 관련 특정 역의 창설 날짜는 언제야?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  

연관 문서: 문서4  
질문: 서울 지하철 9호선에서 높은 환승율을 보이는 노선은 뭐야?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  

연관 문서: 문서5  
질문: 9호선 1단계 구간의 시공 자동차 제조 회사는 어디야?  
판단: 해당 질문은 5개의 문서를 모두 사용해도 답변이 불가능합니다. => True  


In [ ]:
import re

def remove_false_decisions(text):
    # 각 질문-판단 세트를 찾는 정규표현식
    pattern = r'(연관 문서: 문서\d+\n질문: .+?\n판단: .+? => (?:True|False))'

    # 모든 세트를 찾아서 리스트로 저장
    sets = re.findall(pattern, text, re.DOTALL)

    # True인 세트만 필터링
    filtered_sets = [s for s in sets if "=> True" in s]

    # 필터링된 세트를 다시 하나의 문자열로 결합
    result = "\n\n".join(filtered_sets)

    return result

In [ ]:
test_result = remove_false_decisions(df['no_answer_question'].to_list()[15])
print(test_result)

In [ ]:
def extract_questions(text):
    # 질문을 찾는 정규표현식 패턴
    pattern = r'질문: (.*?)(?=\n판단:)'

    # 모든 질문을 찾아서 리스트로 저장
    questions = re.findall(pattern, text, re.DOTALL)

    # 질문에서 앞뒤 공백 제거
    questions = [q.strip() for q in questions]

    return questions

In [ ]:
# 결과 출력
questions_list = extract_questions(test_result)
print(questions_list)

[]


In [ ]:
def select_random_question(questions_list):
    if not questions_list:
        return None  # 리스트가 비어있을 경우 None 반환
    return random.choice(questions_list)

In [ ]:
one_questions = []
for five_questions in df['no_answer_question'].to_list():
  true_questions = remove_false_decisions(five_questions)
  questions_list = extract_questions(true_questions)
  random_question = select_random_question(questions_list)
  one_questions.append(random_question)

In [ ]:
one_questions[:5]

[None, None, None, None, None]

In [ ]:
# 5개의 문서를 모두 사용해도 답을 찾을 수 없는 질문
df['no_answer_question'] = one_questions

In [ ]:
# 형식 등의 사유로 질문이 추출되지 않았을 경우
df = df.dropna(subset='no_answer_question').reset_index(drop=True)

In [ ]:
one_questions = df['no_answer_question'].to_list()

## 답변 생성

In [ ]:
system_prompt = """당신은 검색된 문서부터 질문의 답변을 작성하는 언어 모델입니다.

### 지시사항
1. 질문으로부터 주어진 다수의 문서에서 답을 찾아 작성하세요.
2. 검색된 문서에 질문의 답이 없는 경우에는 질문에 대한 내용을 언급하면서 답을 찾을 수 없다고 작성하세요.
3. 답변에서 참고자료의 내용을 인용한 경우, 답변 맨 끝에 인용한 문서 id를 추가하세요. 답변에 내용을 인용한 경우에만 작성해야 합니다. 인용한 게 없다면 쓰지마세요.
4. 문서 id 값은 검색 결과에서 가져온 것이며 두 개의 대괄호로 묶어야 하며 ref라는 키워드와 함께 작성해야 합니다. 예를 들어 특정 문단이 3번, 5번 문서에서 인용했다면 다음과 같이 작성하십시오.
예시) ~~~해서 했습니다. [[ref3]], [[ref5]]
5. 주요 문장 또는 문단 뒤에 ref문서번호를 덧붙이는 것은 매우 중요합니다. 이는 반드시 지켜져야 합니다.
6. 문단 중간 중간에 인용이 들어가더라도 답변은 하나의 이어지는 평문으로 작성하십시오.
7. 주어진 문서는 검색 결과입니다. 따라서 질문과 연관없는 문서가 숨어있을 수 있으니 이를 인용하지 않도록 주의하십시오.
8. 검색 결과에 없는 내용은 답변으로 작성하지 마십시오. 가장 중요한 지시사항입니다. 따라서 인용 없이는 그 어떠한 설명도 제공할 수 없습니다.
9. 답변은 최대한 풍부하게 작성해주시기 바랍니다.
10. 단, 질문에 대한 답이 검색 결과에 없다면 다음과 같이 답변을 작성하십시오. 반드시 아래의 형식을 따라야 합니다.
=> 답변 형식: 검색 결과에는 ~~의 반응을 찾을 수 없습니다.

예시를 드리겠습니다.

예시)
질문: 원종 설립 당시 일본 정부의 반응은 어떠했나요?
답변: 검색 결과에는 원종 설립 당시 일본 정부의 반응을 찾을 수 없습니다.

질문: 중국의 던전앤파이터 인기 게임 순위는 어떻게 되나요?
답변: 검색 결과에는 중국의 던전앤파이터 인기 게임 순위를 찾을 수 없습니다.

검색 결과에 질문에 대한 답변이 없다면 위 형식을 반드시 따르십시오.
"""

In [ ]:
samples = []
for context, negative_samples in zip(df['context'].to_list(), df['negative_samples'].to_list()):
  samples.append([context] + list(negative_samples))

In [ ]:
# 프롬프트에 검색 문서 넣는 연습
for idx, s in enumerate(samples[5]):
  print('문서' + str(idx + 1) + ':' + s)
  print('---')

문서1:정유사업 의존도가 높은 GS그룹이 STX에너지를 인수한다. 앞으로 발전사업 강화를 통해 에너지 전문기업으로 도약하는 계기를 마련한 것이라는 평가가 나온다. GS그룹의 자원개발, 신재생에너지, 해외 발전사업 등에도 속도가 붙을 전망이다.GS그룹은 일본 오릭스가 (주)GS-LG상사 컨소시엄을 STX에너지 경영권 매각을 위한 배타적 협상자로 확정했다고 11일 발표했다. 컨소시엄은 STX에너지의 최대주주인 오릭스 보유지분 96.35% 중 72%를 인수할 예정이다. 인수가 끝나면 GS그룹 지주사인 (주)GS가 STX에너지 최대주주로 올라선다. GS그룹은 정유와 석유화학 위주의 사업구조를 바꾸기 위해 지난해와 올해 적극적인 인수합병(M&A) 에 나섰으나 뜻을 이루지 못하다, LG상사와 손잡을 만큼 큰 의욕을 보인 STX에너지 인수전을 통해 마침내 반전을 이뤄냈다. 앞서 GS리테일은 지난해 웅진코웨이 입찰에 뛰어들었다가 막판에 탈락했고, 올해는 GS에너지가 웅진케미칼 인수에 실패했다.GS그룹은 지난해 총 매출(73조원) 가운데 약 66%인 47조8700억원을 주력 계열사인 GS칼텍스가 책임졌을 정도로 정유와 석유화학 비중이 크다. GS는 이에 따라 신성장 동력을 찾는 데 많은 힘을 쏟아왔다.그룹 관계자는 “지난 9월 웅진케미칼 인수에 실패한 후 그룹 차원에서 STX에너지 인수를 위해 상당한 공을 들였다”며 “그룹의 에너지 관련 계열사들과 상당한 시너지 효과가 기대된다”고 말했다.
---
문서2:신세계가 서울 강남 쇼핑타운 조성에 본격 나섰다. 지난해 강남점이 세들어 있는 센트럴시티를 인수한 데 이어 서울고속버스터미널까지 손에 넣으며 이 일대를 ‘반포 신세계타운’으로 만든다는 계획이다. ▶본지 4월1일자 A1, 6면 참조신세계가 최대주주로 있는 센트럴시티는 ‘코에프씨 IBKS-케이스톤 기업재무안정 PEF(사모펀드)’로부터 서울고속버스터미널 지분 38.7%를 2200억원에 인수했다고 1일 밝혔다. 이번 지분 인수로 센트럴시티는 서울고속버스터미널의 최대주주가 됐다. 신세

In [ ]:
user_prompts = []
for sample in samples:
  user_prompt = '검색 결과:\n'
  for idx, s in enumerate(sample):
    user_prompt = user_prompt  + '문서' + str(idx + 1) + ':' + s + '\n'
    user_prompt = user_prompt + '---\n'
  user_prompts.append(user_prompt)

In [ ]:
len(user_prompts)

146

In [ ]:
user_prompt_for_answer = []
for multi_document_question, user_prompt in zip(one_questions, user_prompts):
  user_prompt_for_answer.append('질문:' + multi_document_question + '\n' + user_prompt + '답변:')

In [ ]:
print(user_prompt_for_answer[0])

질문:한화호텔&리조트에서 제공하는 중식 메뉴는 뭐니?
검색 결과:
문서1:현대·기아자동차가 해외 시장에서의 판매 호조에 힘입어 올 상반기 세계 시장에서 404만대를 판매했다. 올해 초 설정한 연간 목표 786만대를 넘어 처음으로 800만대 돌파도 기대된다.현대차와 기아차, 한국GM, 쌍용차, 르노삼성 등 국내 5개 완성차 업체들은 올 상반기 글로벌 시장에서 모두 450만7665대를 판매한 것으로 1일 집계됐다. 작년 같은 기간보다 3.2% 늘어난 수치다. 국내 판매는 70만7368대(5.1% 증가), 해외 판매는 380만304대(2.9% 증가)로 나타났다.현대차는 국내 34만여대, 해외 214만여대 등 모두 249만여대를 판매해 전년 동기 대비 4.4% 늘었다. 국내에서는 신형 쏘나타와 신형 제네시스, 스포츠유틸리티차량(SUV)들이 선전하며 6.4% 성장을 이끌어냈다.국내 시장에서는 지난 3월 말 출시한 신형 LF(2만9153대)를 포함해 쏘나타 모델이 5만4785대 팔려 전년 동기 대비 18.1% 늘었다. 작년 12월 나온 신형 제네시스는 2만380대로 지난해 상반기 6509대보다 세 배 늘었다.기아차는 주력 모델 노후화 탓에 국내 판매가 21만8764대로 3.4% 감소했다. 2010년 4월 나온 K5는 2만5603대(8.9% 감소)가 팔려 동급인 쏘나타의 절반에도 못 미쳤다. 기아차는 그러나 해외 시장에서는 국내 5개 업체 가운데 가장 높은 9% 성장률을 기록했다. 회사 관계자는 “쏘울과 K3 등이 미국과 유럽에서 선전하고 있다”고 설명했다.현대차와 기아차는 상반기 합계 404만3405대를 판매해 반기 기준 처음으로 400만대를 돌파했다. 연초 설정한 연간 판매 목표 786만대의 51.2%를 달성해 그룹 최초 800만대 달성도 가시권에 들어왔다. 다만 하반기에는 통상임금 확대 등을 주장하는 노조가 파업 가능성을 내비치고 있어 생산 차질이 빚어질 수 있다는 게 걸림돌이다.국내 자동차업계 3위 한국GM은 국내 시장에서 7만1958대를 팔아 2003년 7만473

In [ ]:
# 5개로 테스트
result_lst = []
for user_prompt in tqdm(user_prompt_for_answer[:5]):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '답변:'}
      ],
    temperature=0
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 5/5 [00:05<00:00,  1.17s/it]


In [ ]:
result_lst

['검색 결과에는 한화호텔&리조트에서 제공하는 중식 메뉴를 찾을 수 없습니다.',
 '검색 결과에는 이준용이 일본에서 어떤 특별한 교육을 받았는지에 대한 내용을 찾을 수 없습니다.',
 '검색 결과에는 에이어즈의 첫 번째 앨범 제목을 찾을 수 없습니다.',
 '검색 결과에는 몰디브 관광청의 세부적 비즈니스 계획을 찾을 수 없습니다.',
 '검색 결과에는 강은희 후부 장관 후보의 대구 외 인생 이야기를 찾을 수 없습니다.']

In [ ]:
# 전체 데이터에 대해서 답변을 생성
result_lst = []
for user_prompt in tqdm(user_prompt_for_answer):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt + '답변:'}
      ]
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 146/146 [03:24<00:00,  1.40s/it]


In [ ]:
df['no_answer'] = result_lst

In [ ]:
# 답이 없기를 기대했지만 추출된 경우
df[df['no_answer'].str.contains('ref')]

,title,news_category,source,context,question,question_type,is_impossible,answer_text,answer_start,negative_samples,no_answer_question,no_answer
10,"정부, 기업이 원하는 인재 공급 못하면 지원 중단",종합,hankyung,산업계가 요구하는 인재를 공급하는 민간 산·학·연 협의체 가운데 우수한 곳은 업종 ...,NCS에 맞게 개편될 인적자원개발협의체의 수는?,2,False,24개,719,[정부가 올해부터 5·7·9급 국가직 공무원 채용시험에 국가직무능력표준(NCS)을 ...,전경련의 회장단 영입 방식은 어떻게 될까?,전경련의 회장단 영입 방식은 회원사 범위를 넓히고 새로운 기업을 영입하는 방식을 채...
36,"e스포츠 공정위원회, 정식 발족 및 e스포츠 종목 3사와 협약 체결",게임,acrofan,e스포츠 공정위원회가 지난 6월 30일(화) e스포츠 명예의 전당(서울 마포)에서 ...,1월 이후에 새로 e스포츠 공정위원이 된 인물의 이름은?,2,False,강범현,615,[한국e스포츠협회가 운영하는 e스포츠 명예의 전당(서울 마포)은 오는 24일(토) ...,"지스타에 참가하기로 결정했을 때, 게임업계의 반발은 어떤 형태로 나타났나요?",게임업계는 지스타에 참가하기로 결정했을 때 반발의 형태로 불참 의사를 표명했습니다....
42,과도한 압박은 문제있어”,경제금융,hankyung,박병원 은행연합회장(사진)이 최근 중징계를 받은 김종준 하나은행장의 거취 문제를 놓...,김종준의 금융권 재취업이 제한된 계기가 된 사건 당시 그의 직위는?,2,False,하나캐피탈 사장,265,[김용준 국무총리 후보자(75·사진)는 장애를 극복한 입지전적인 인물이다. 세 살 ...,박병원이 어떤 방식으로 금융당국을 비판했는지 아는 사람 있어?,박병원 은행연합회장은 최근 기자간담회에서 금융당국의 압박에 대해 비판적인 입장을 표...
48,석유화학·전자·화장품 업종도 ‘인력 빼가기’ 심각,종합,hankyung,중국 업체들의 한국 인력 빼가기는 다른 업종에서도 나타나고 있다. 중국 업체들은 헤...,중국으로 이직한 엔지니어가 이직 전 한국에서 받던 돈은?,1,False,연봉 4000만원,480,[“중국 내 인건비가 매년 18%가량 상승하면서 현지 공장 운영이 한계에 이르렀습니...,중국 기업이 한국에서 어떤 방식으로 인력을 가져가는지 구체적으로 말해줘.,중국 기업들은 한국에서 인력을 가져가는 다양한 방식으로 활동하고 있습니다. 주로 헤...
66,"64억4000만원 ...이태원 2층 벽돌집, 2년째 단독주택값 1위",부동산,hankyung,올해 전국 표준단독주택 중 공시가격이 가장 높은 집은 서울 이태원동 이태원로 27길...,전국에서 가장 비싼 단독주택의 가격은?,2,False,149억원,548,[국토교통부가 전국 3178만필지를 대상으로 올해 개별공시지가를 조사했다. 토지가 ...,대구 단독주택 가격이 오르는 이유가 뭔지 혹시 알아?,대구 단독주택 가격이 오르는 이유로는 여러 가지 요소가 지적될 수 있습니다. 최근 ...
71,동방성련선 ~ Undefined Fantastic Object,None,wikipedia,"지하에 봉인되었던 누에. 인간들에게 모습을 드러내지 않고, 인간이 자신을 무서워하는...","『성련선』 내용에 따르면, 대전시에서 UFO로 인식되어지는 물체는?",1,False,"""정체불명의 씨앗""",601,[2000년 11월 22일경 김옥경은 조부이자 애국지사인 김용원의 유적비 건립 현황...,엑시톤의 발견은 어떤 과학적 이론을 기반으로 연구되었나요?,엑시톤의 발견은 2차원 자성 물질에서 나타나는 독특한 양자 현상에 기초하여 진행된 ...
93,"캡틴은 메시, 경기 상대는 외계인 ...삼성 축구팀 ‘갤럭시 11’의 정체는 …",산업섹션,hankyung,외계인과 지구대표팀의 축구시합을 예고한 삼성전자의 갤럭시 유튜브 동영상이 화제다. ...,갤럭시 11' 캠페인의 첫 영상이 업로드된 날은 며칠인가?,2,False,4일,49,[대성마이맥에서 지난 3일에 공개한 광고 영상이 일주일 만에 조회 수 495만 뷰를...,삼성전자가 어벤져스와 어떤 특별한 계약을 맺었는지 알려줘.,삼성전자는 마블과 협력하여 어벤져스 시리즈에 대한 마케팅 파트너십을 맺었습니다. 이...
105,남산 자락에서...신나게 즐기는...우리음악 축제,문화/TV,hankyung,"지난해 평균 객석점유율 121%를 기록한 국립극장의 ‘여기, 우리 음악이 있다(이하...",여우락 축제의 객석이 채워진 비율은?,1,False,121,13,[중국 국경절 연휴(1~7일)와 ‘코리아 블랙프라이데이’를 맞아 중국인 관광객(요우...,여우락 페스티벌의 기획 의도가 뭐였는지 알아?,여우락 페스티벌의 기획 의도는 전통 음악에 뿌리를 두고 동시대와 호흡하며 창작 활동...
129,일찍 찾아온 여름 … 에어컨·제습기 ‘씽씽’,생활경제,hankyung,이른 더위 영향으로 에어컨 판매가 큰 폭으로 늘었다.롯데백화점은 올 들어 지난달까지...,현대백화점에서 에어컨을 사고 5% 상품권을 받을 수 있는 마지막 날은?,1,False,오는 11일,397,[백화점들이 신년 세일 마지막 주말 매출을 끌어올리기 위해 총력전을 펼친다.현대백화...,이번 겨울 패딩 판매 부진의 주된 영향은 뭐라고 생각해?,올해 겨울 패딩 판매 부진의 주된 영향은 예상보다 따뜻한 날씨 때문입니다. 기상청은...
133,에드워드 코드링턴,None,wikipedia,"1826년 12월 코드링턴은 지중해 함대의 총사령관으로 임명되었고, 1827년 2월...",코드링턴이 메메트 알리와 조우한 것은 몇 년인가?,2,False,1828년,642,"[프랑스와의 적대관계가 재개되자 그는 순양함에서 남아있었고, 1805년 봄 전열함 ...",게이츠와 앨런이 처음 만든 소프트웨어의 이름은 뭐였어?,게이츠와 앨런이 처음 만든 소프트웨어의 이름은 알테어 BASIC입니다. 그들은 19...


답이 없어야 하는데 추출된 경우에는 삭제합니다. (데이터를 보고 이상하지 않다고 판단되면 사용하셔도 됩니다.)

In [ ]:
df = df[~df['no_answer'].str.contains('ref')].reset_index(drop=True)

In [ ]:
# 이상한 데이터가 없는지 체크
sample_check = []
for sample in df['no_answer'].to_list():
  if '검색 결과' not in sample:
    sample_check.append(sample)

In [ ]:
len(sample_check)

0

In [ ]:
# sample_check[3]

In [ ]:
samples = []
for context, negative_samples in zip(df['context'].to_list(), df['negative_samples'].to_list()):
  samples.append([context] + list(negative_samples))

In [ ]:
print('질문:', df['no_answer_question'].loc[15])

질문: MMA WEEK에서 어떤 신인 아티스트들이 모두 공연할 거야?


In [ ]:
for idx, s in enumerate(samples[15]):
  print('문서' + str(idx + 1) + ':' + s)
  print('---')

문서1:라이엇 게임즈의 1인칭 전술 슈팅 게임(이하 FPS) 발로란트가 연말 시즌을 맞아 신규 모드와 한정 스킨 등 풍성한 업데이트를 선보인다. 발로란트는 크리스마스와 연말연시를 기념해 눈싸움 모드를 마련했다. 해당 모드는 5:5 팀 데스매치로 날아오는 눈덩이를 피하고 적에게 눈덩이를 발사하는 방식의 전투다. 킬조이 요원이 브림스톤의 화염병 발사기를 개조해 개발한 ‘눈덩이 발사기’가 공통 무기로 사용되며 먼저 50킬을 달성하는 팀이 승리한다. 눈덩이는 무제한으로 생성되며 유일한 근접 무기는 칼이다. 아이템은 맵 곳곳에서 ‘선물’이 되어 등장한다. 선물로 제공되는 아이템은 눈덩이를 빠르게 발사할 수 있는 ‘속사’, 눈덩이가 커지는 ‘설상가상’, 눈덩이가 바닥과 벽에 닿으면 반사되는 ‘도탄’, 스케이트를 타고 이동 할 수 있는 ‘스케이트’ 등이 있으며, 독톡한 강화 효과를 더해주고 또 팀원들과도 나눠서 사용할 수도 있다. 게임 종료 시까지 계속해서 부활할 수는 있지만, 강화 효과를 떨어뜨려 다른 플레이어에게 빼앗길 수도 있다. 눈싸움 모드는 이벤트 콘텐츠로 한시적으로 제공된다. 이달 15일부터 29일까지 약 2주간 즐길 수 있으며 플레이 한 유저는 ‘눈사람 친구 총기 장식’을 획득할 수 있다. 발로란트는 유럽 시골 마을의 겨울 풍경을 담은 무기 스킨 ‘윈터원더랜드’도 선보인다. 신규 스킨은 밝고 평화로운 느낌을 담은 디자인으로 게임 내에서 크리스마스와 연말 분위기를 한층 더할 예정이다. 윈터원더랜드 스킨은, 장착한 채로 맵 내 어두운 곳에 위치해 있으면 겨울 풍경이 화려한 밤 풍경으로 변하는 독특한 효과가 포함돼 있다. 적용 무기는 밴달, 팬텀, 고스트, 마샬과 근접무기이며 모든 옵션이 포함된 세트 상품은 5,100 발로란트 포인트(VP)로 구매할 수 있다. 오는 11일부터 내년 1월 12일까지 특별한 할인 혜택도 제공된다. 발로란트 상점 내 등장하는 ‘야시장’에서는 무기 스킨을 최대 50% 할인 가격으로 선보일 예정이다. 할인 대상은 최대 6가지의 셀렉트, 디럭

In [ ]:
print(df['no_answer'].loc[15])

검색 결과에는 MMA WEEK에서 어떤 신인 아티스트들이 모두 공연할 것인지에 대한 내용을 찾을 수 없습니다.


In [ ]:
import re

In [ ]:
# 인용 문서 번호 추출
def extract_ref_numbers(text):
    # 정규표현식을 사용하여 [[ref숫자]] 패턴 찾기
    pattern = r'\[\[ref(\d+)\]\]'

    # 모든 매치 찾기
    matches = re.findall(pattern, text)

    # 문자열을 정수로 변환하고 중복 제거 후 정렬
    unique_numbers = sorted(set(map(int, matches)))

    return unique_numbers

In [ ]:
# 테스트
result = extract_ref_numbers(df['no_answer'].loc[15])
print(result)

[]


In [ ]:
extracted_ref_numbers = []
for sample in df['no_answer'].to_list():
  extracted_ref_numbers.append(extract_ref_numbers(sample))

In [ ]:
df['extracted_ref_numbers'] = extracted_ref_numbers

In [ ]:
# 인용 문서가 존재하는지 확인
df[~(df['extracted_ref_numbers'].apply(str) == '[]')].reset_index(drop=True)

,title,news_category,source,context,question,question_type,is_impossible,answer_text,answer_start,negative_samples,no_answer_question,no_answer,extracted_ref_numbers


In [ ]:
df['search_result'] = samples

In [ ]:
dataset = datasets.Dataset.from_pandas(df)

In [ ]:
dataset

Dataset({
    features: ['title', 'news_category', 'source', 'context', 'question', 'question_type', 'is_impossible', 'answer_text', 'answer_start', 'negative_samples', 'no_answer_question', 'no_answer', 'extracted_ref_numbers', 'search_result'],
    num_rows: 136
})

In [ ]:
from google.colab import userdata
HF_WRITE_API_KEY = userdata.get('HF_WRITE_API_KEY')

login(HF_WRITE_API_KEY)

In [ ]:
dataset.push_to_hub("klue-mrc-no-answer-question")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/longjae/klue-mrc-no-answer-question/commit/741ba26b3659254656de2f1153c4cf7ec1a5a2fe', commit_message='Upload dataset', commit_description='', oid='741ba26b3659254656de2f1153c4cf7ec1a5a2fe', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/longjae/klue-mrc-no-answer-question', endpoint='https://huggingface.co', repo_type='dataset', repo_id='longjae/klue-mrc-no-answer-question'), pr_revision=None, pr_num=None)

In [ ]:
df[['no_answer_question', 'search_result', 'no_answer', 'extracted_ref_numbers']]

,no_answer_question,search_result,no_answer,extracted_ref_numbers
0,한화호텔&리조트에서 제공하는 중식 메뉴는 뭐니?,[현대·기아자동차가 해외 시장에서의 판매 호조에 힘입어 올 상반기 세계 시장에서 4...,검색 결과에는 한화호텔&리조트에서 제공하는 중식 메뉴를 찾을 수 없습니다.,[]
1,이준용이 일본에서 어떤 특별한 교육을 받았는지 알아?,[일본 망명 직후부터 이준용은 귀인 엄씨의 빈 책봉을 반대하고 나섰다. 이어 고종은...,검색 결과에는 이준용이 일본에서 어떤 특별한 교육을 받았는지 찾을 수 없습니다.,[]
2,에이어즈의 첫 번째 앨범 제목이 뭐였는지 알아?,[뮤지션의 일대기를 다룬 잘 만든 한 편의 음악 영화를 보는 듯했다. 음악에 착 달...,검색 결과에는 에이어즈의 첫 번째 앨범 제목을 찾을 수 없습니다.,[]
3,몰디브 관광청의 세부적 비즈니스 계획을 어떤 자료에서 확인할 수 있어?,[야놀자 계열사 여행대학(총장 정상근)이 오는 18일(월)까지 ‘시니어 꿈꾸는 여행...,검색 결과에는 몰디브 관광청의 세부적 비즈니스 계획을 찾을 수 없습니다.,[]
4,강은희 후부 장관 후보의 대구 외 인생 이야기는 뭐야?,[박근혜 대통령이 21일 총선에 출마하는 현직 장관 다섯 명과 임기가 만료된 국민권...,검색 결과에는 강은희 후부 장관 후보의 대구 외 인생 이야기를 찾을 수 없습니다.,[]
...,...,...,...,...
131,이슬람국가(IS)가 이라크 외부에서 어떤 영향을 미칠 것인지 말해줄래?,[이란의 핵무기 개발을 놓고 대립해온 미국과 이란 간 ‘화해 모드’가 조성되고 있다...,검색 결과에는 이슬람국가(IS)가 이라크 외부에서 어떤 영향을 미칠 것인지에 대한 ...,[]
132,서울둘레길을 처음 제안한 사람은 누구인지 얘기해줄래?,"[여의도는 원래 불모지로, 군사 훈련장으로 쓰이기도 했다. 일제는 표면상으로는 민간...",검색 결과에는 서울둘레길을 처음 제안한 사람에 대한 정보를 찾을 수 없습니다.,[]
133,화재가 발생했을 때 사람들은 대개 어떤 행동을 취하나요?,[볼펜은 일회용 모델과 리필 모델로 생산된다. 리필을 사용하면 볼펜 및 소켓을 포함...,검색 결과에는 화재가 발생했을 때 사람들의 일반적인 행동을 찾을 수 없습니다.,[]
134,뇌전증 환자가 틱톡을 사용하면 어떤 문제가 생길까?,"[간질(癎疾) 또는 전간(癲癎)이라고도 하며, 과거에는 간질이 정식 명칭으로 사용되...",검색 결과에는 뇌전증 환자가 틱톡을 사용하면서 겪을 수 있는 문제를 찾을 수 없습니다.,[]
